In [3]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [10]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [6]:
#Tokenizing
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [11]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

117 documents
30 classes ['Identity', 'activity', 'age', 'appreciate', 'contact', 'covid19', 'cricket', 'datetime', 'exclaim', 'goodbye', 'google', 'greeting', 'greetreply', 'haha', 'inspire', 'insult', 'jaskaran', 'jokes', 'news', 'nicetty', 'no', 'options', 'programmer', 'riddle', 'song', 'suggest', 'thanks', 'timer', 'weather', 'whatsup']
138 unique lemmatized words ["'m", "'s", ',', '10', '19', 'a', 'age', 'am', 'anyone', 'are', 'ask', 'awesome', 'bad', 'bbye', 'be', 'best', 'bi', 'bye', 'ca', 'can', 'contact', 'could', 'covid', 'creator', 'cricket', 'current', 'date', 'day', 'designed', 'developer', 'do', 'doing', 'dumb', 'fine', 'for', 'fuck', 'funny', 'get', 'good', 'goodbye', 'google', 'great', 'haha', 'he', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'hot', 'how', 'i', 'idiot', 'india', 'inspiration', 'inspires', 'internet', 'is', 'it', 'jaskaran', 'joke', 'know', 'later', 'latest', 'laugh', 'lmao', 'lol', 'lost', 'made', 'make', 'match', 'me', 'motivates', "n'

In [12]:
# Creating training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [13]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('leo_model.h5', hist)


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/200
24/24 [==============================] - 1s 2ms/step - loss: 3.4302 - accuracy: 0.0427 
Epoch 2/200
24/24 [==============================] - 0s 2ms/step - loss: 3.3767 - accuracy: 0.0256
Epoch 3/200
24/24 [==============================] - 0s 2ms/step - loss: 3.2990 - accuracy: 0.0940
Epoch 4/200
24/24 [==============================] - 0s 2ms/step - loss: 3.2285 - accuracy: 0.1026
Epoch 5/200
24/24 [==============================] - 0s 2ms/step - loss: 3.2327 - accuracy: 0.1111
Epoch 6/200
24/24 [==============================] - 0s 2ms/step - loss: 3.1629 - accuracy: 0.1368
Epoch 7/200
24/24 [==============================] - 0s 2ms/step - loss: 3.1017 - accuracy: 0.1538
Epoch 8/200
24/24 [==============================] - 0s 2ms/step - loss: 2.9658 - accuracy: 0.2222
Epoch 9/200
24/24 [==============================] - 0s 2ms/step - loss: 2.8942 - accuracy: 0.2222
Epoch 10/200
24/24 [==============================] - 0s 2ms/step - loss: 2.8431 - accuracy: 0.2051
Epoch 11